## Libraries

In [1]:
import pandas as pd
import json

import numpy as np

from scipy.stats import ttest_rel
from itertools import combinations

from scipy.stats import wilcoxon  

/home/p0l3/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Data Loading

In [2]:
# Read the CSV (handle missing headers)
df = pd.read_csv("CLIMABENCH.csv", delimiter=",", skip_blank_lines=True, dtype=str)

# Convert all comma-style decimals (e.g., "0,81008") to dots (e.g., "0.81008")
df = df.map(lambda x: x.replace(",", ".") if isinstance(x, str) else x)

# Fill missing MODEL names (assuming it's only missing in seed rows)
df["MODEL"].fillna(method="ffill", inplace=True)

# Drop completely empty rows
df.dropna(how="all", inplace=True)

# Extract relevant columns: model, seed, task results
models = df["MODEL"].unique().tolist()
tasks = [col for col in df.columns if col not in ["MODEL", "SEED", "TOTAL AVG"] and "AVG" not in col]

# Convert data into nested dictionary
results = {}

for _, row in df.iterrows():
    model = row["MODEL"]
    seed = row["SEED"]
    
    if model not in results:
        results[model] = {}

    for task in tasks:
        score = row[task]
        avg_col = f"AVG {task}"

        if pd.isna(score):  # Skip missing values
            continue

        score = float(score)  # Convert string to float
        
        if task not in results[model]:
            results[model][task] = {"seeds": [], "average": None}

        results[model][task]["seeds"].append(score)

        # Assign average score (only from rows where it's present)
        if pd.notna(row[avg_col]):
            results[model][task]["average"] = float(row[avg_col])

# Convert to JSON and save
json_output = json.dumps(results, indent=4)

with open("parsed_results.json", "w") as f:
    f.write(json_output)

# print(json_output)

/tmp/ipykernel_6602/882887662.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["MODEL"].fillna(method="ffill", inplace=True)
/tmp/ipykernel_6602/882887662.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["MODEL"].fillna(method="ffill", inplace=True)


In [3]:
results["CliReBERT"]["ClimateInsurance"]

{'seeds': [0.81008, 0.81912, 0.81279, 0.81765, 0.80554], 'average': 0.813036}

In [4]:
for model in results:
    print(model)

CliReBERT
CliSciBERT
SciBERT
SciClimateBERT
ClimateBERT
BERT
RoBERTa
DistilRoBERTa


In [5]:
# **ASSERTION CHECK** - Compare provided average with calculated average
inconsistencies = []

for model, tasks in results.items():
    for task, data in tasks.items():
        if data["average"] is not None and data["seeds"]:
            computed_avg = sum(data["seeds"]) / len(data["seeds"])
            if round(computed_avg, 6) != round(data["average"], 6):  # Allow small rounding differences
                inconsistencies.append({
                    "model": model,
                    "task": task,
                    "provided_average": data["average"],
                    "computed_average": computed_avg,
                    "seeds": data["seeds"]
                })

# Print inconsistencies
if inconsistencies:
    print("❌ Inconsistencies found!")
    for issue in inconsistencies:
        print(f"Model: {issue['model']}, Task: {issue['task']}")
        print(f"  - Provided Average: {issue['provided_average']}")
        print(f"  - Computed Average: {issue['computed_average']}")
        print(f"  - Seeds: {issue['seeds']}")
else:
    print("✅ All averages match the computed values!")

✅ All averages match the computed values!


## Table 1

In [6]:
print(results.keys())
print(results["CliReBERT"].keys())
print(results["CliReBERT"]["CDPCities"].keys())
print(results["CliReBERT"]["CDPCities"]["average"], results["CliReBERT"]["CDPCities"]["seeds"])

dict_keys(['CliReBERT', 'CliSciBERT', 'SciBERT', 'SciClimateBERT', 'ClimateBERT', 'BERT', 'RoBERTa', 'DistilRoBERTa'])
dict_keys(['ClimateInsurance', 'CDPCities', 'ClimateText', 'ClimateStance', 'ClimateEng', 'SciDCC', 'ClimateFEVER'])
dict_keys(['seeds', 'average'])
0.618628 [0.61758, 0.63117, 0.61601, 0.6114, 0.61698]


In [7]:
# --- 1. Flatten the nested 'results' dictionary into a list of records ---
records = []
for model, datasets in results.items():
    for dataset, metrics in datasets.items():
        seeds = metrics['seeds']
        records.append({
            "model": model,
            "dataset": dataset,
            "mean": np.mean(seeds),
            "std": np.std(seeds, ddof=1),
        })

df = pd.DataFrame(records)

# --- 1.5. Rename the specific model (NEW STEP) ---
df['model'] = df['model'].replace({'CliReBERT_clirevocab_10e_91024': 'CliReBERT'})


# --- 2. Create summary table with MultiIndex columns: ('mean', model), ('std', model) ---
summary = df.pivot(index='dataset', columns='model', values=['mean', 'std'])

# --- 3. Scale only the mean values by 100 to convert to percentages ---
summary_scaled = summary.copy()
summary_scaled['mean'] *= 100
summary_scaled['std'] *= 100

# --- 4. Compute total averages (mean and std) for each model ---
model_means_scaled = summary_scaled['mean'].T
total_avg = model_means_scaled.mean(axis=1)
total_std = summary['mean'].T.std(axis=1, ddof=1) * 100

# --- 5. Format "mean ± std" strings for each dataset/model pair ---
formatted = pd.DataFrame({
    model: summary_scaled['mean'][model].map('{:.2f}'.format) + ' ± ' + summary_scaled['std'][model].map('{:.2f}'.format)
    for model in summary_scaled['mean'].columns
}).T

formatted.index.name = "model"

# --- 6. Add total average column ---
formatted['Total Average'] = total_avg.map('{:.2f}'.format) + ' ± ' + total_std.map('{:.2f}'.format)

# --- 6.5. Reorder the models to the desired specification (NEW STEP) ---
model_order = [
    "RoBERTa",
    "DistilRoBERTa",
    "ClimateBERT",
    "SciClimateBERT",
    "BERT",
    "SciBERT",
    "CliSciBERT",
    "CliReBERT"
]
formatted = formatted.reindex(model_order)

# --- 6.6. Reorder the datasets (columns) (NEW STEP) ---
dataset_order = [
    "ClimateInsurance",
    "CDPCities",
    "ClimateText",
    "ClimateStance",
    "ClimateEng",
    "SciDCC",
    "ClimateFEVER"
]

# Build the final column order, keeping 'Total Average' at the end
# This also handles cases where a dataset might not be in the results
final_column_order = [ds for ds in dataset_order if ds in formatted.columns] + ['Total Average']

formatted = formatted[final_column_order]

# --- 7. Display or save ---
# print(formatted)
formatted.to_csv("table1.csv")
formatted

dataset,ClimateInsurance,CDPCities,ClimateText,ClimateStance,ClimateEng,SciDCC,ClimateFEVER,Total Average
model,,,,,,,,
RoBERTa,79.43 ± 2.56,65.51 ± 0.31,68.62 ± 12.80,39.51 ± 9.39,32.33 ± 23.18,52.55 ± 1.02,63.76 ± 1.55,57.39 ± 16.78
DistilRoBERTa,80.21 ± 1.34,63.85 ± 0.72,73.67 ± 3.77,41.12 ± 1.75,35.32 ± 4.99,52.76 ± 0.77,61.53 ± 1.45,58.35 ± 16.39
ClimateBERT,81.95 ± 0.90,64.21 ± 0.17,73.10 ± 1.10,52.43 ± 4.20,64.51 ± 14.48,52.06 ± 0.69,62.30 ± 0.91,64.37 ± 10.67
SciClimateBERT,79.82 ± 0.95,64.63 ± 0.13,75.64 ± 1.47,40.60 ± 2.76,30.07 ± 4.06,52.56 ± 1.79,61.48 ± 1.07,57.83 ± 18.05
BERT,80.35 ± 1.58,64.42 ± 0.57,82.99 ± 2.44,42.58 ± 1.05,43.91 ± 5.43,52.70 ± 1.32,62.78 ± 1.28,61.39 ± 16.19
SciBERT,81.35 ± 0.96,64.17 ± 0.80,74.00 ± 3.59,39.70 ± 8.12,42.11 ± 15.36,51.75 ± 0.81,62.09 ± 1.32,59.31 ± 15.65
CliSciBERT,80.11 ± 0.88,65.47 ± 0.50,74.93 ± 3.35,41.90 ± 3.66,48.64 ± 2.64,51.93 ± 0.60,60.52 ± 0.76,60.50 ± 14.01
CliReBERT,81.30 ± 0.56,61.86 ± 0.74,84.19 ± 1.11,50.23 ± 2.15,66.04 ± 1.73,53.33 ± 0.68,61.17 ± 1.30,65.45 ± 12.99


## Paired t-test

In [8]:
# Significance level
alpha = 0.05

model_names = list(results.keys())
dataset_names = list(results[model_names[0]].keys())

# Store results in a list of dictionaries for easy conversion to a DataFrame
summary_results = []

# Iterate through all unique pairs of models
for model_a, model_b in combinations(model_names, 2):
    print(f"\n{'='*20} Comparing: {model_a} vs. {model_b} {'='*20}")

    for dataset in dataset_names:
        # Get the lists of scores from the 5 seeds for each model
        scores_a = results[model_a][dataset]['seeds']
        scores_b = results[model_b][dataset]['seeds']
        
        # Get the average scores for comparison
        avg_a = results[model_a][dataset]['average']
        avg_b = results[model_b][dataset]['average']

        # Perform the paired t-test
        t_statistic, p_value = ttest_rel(scores_a, scores_b)
        print(t_statistic)
        # Determine which model is better if the difference is significant
        winner = "None"
        is_significant = p_value < alpha
        if is_significant:
            winner = model_a if avg_a > avg_b else model_b
        
        # Store for summary table
        summary_results.append({
            "Model A": model_a,
            "Model B": model_b,
            "Dataset": dataset,
            "P-Value": p_value,
            "Significant (p < 0.05)": is_significant,
            "Winner": winner,
            f"{model_a} Avg": avg_a,
            f"{model_b} Avg": avg_b,
        })

        # --- Print detailed interpretation for each test ---
        print(f"\n--- On Dataset: {dataset} ---")
        print(f"  {model_a} average: {avg_a:.4f}")
        print(f"  {model_b} average: {avg_b:.4f}")
        print(f"  P-value: {p_value:.4f}")

        if is_significant:
            print(f"  ✅ Result: The difference is statistically significant.")
            print(f"     Winner: {winner} is significantly better on this dataset.")
        else:
            print(f"  ❌ Result: The difference is NOT statistically significant.")
            print(f"     Conclusion: We cannot claim one model is better than the other on this dataset.")


# --- Create a Summary DataFrame ---
summary_df = pd.DataFrame(summary_results)
print("\n\n" + "="*60)
print("                  STATISTICAL TEST SUMMARY TABLE")
print("="*60)
print(summary_df)


==================== Comparing: CliReBERT vs. CliSciBERT ====================
2.505754939658617

--- On Dataset: ClimateInsurance ---
  CliReBERT average: 0.8130
  CliSciBERT average: 0.8011
  P-value: 0.0664
  ❌ Result: The difference is NOT statistically significant.
     Conclusion: We cannot claim one model is better than the other on this dataset.
-10.187539084031885

--- On Dataset: CDPCities ---
  CliReBERT average: 0.6186
  CliSciBERT average: 0.6547
  P-value: 0.0005
  ✅ Result: The difference is statistically significant.
     Winner: CliSciBERT is significantly better on this dataset.
5.618819348581148

--- On Dataset: ClimateText ---
  CliReBERT average: 0.8419
  CliSciBERT average: 0.7493
  P-value: 0.0049
  ✅ Result: The difference is statistically significant.
     Winner: CliReBERT is significantly better on this dataset.
5.6246847124502

--- On Dataset: ClimateStance ---
  CliReBERT average: 0.5023
  CliSciBERT average: 0.4190
  P-value: 0.0049
  ✅ Result: The differe

In [9]:
# --- Analysis with Bonferroni Correction ---

# 1. Count total number of tests
num_model_pairs = len(list(combinations(model_names, 2)))
num_datasets = len(dataset_names)
total_tests = num_model_pairs * num_datasets
original_alpha = 0.05
bonferroni_alpha = original_alpha / total_tests

print(f"\n\n{'='*60}")
print("           ANALYSIS WITH BONFERRONI CORRECTION")
print(f"{'='*60}\n")
print(f"Original alpha: {original_alpha}")
print(f"Total number of tests: {total_tests}")
print(f"Bonferroni-corrected alpha: {bonferroni_alpha:.6f}\n")


# Re-evaluate the summary DataFrame with the new alpha
summary_df['Significant (Bonferroni)'] = summary_df['P-Value'] < bonferroni_alpha
summary_df['Winner (Bonferroni)'] = summary_df.apply(
    lambda row: row['Winner'] if row['Significant (Bonferroni)'] else 'None', 
    axis=1
)

# Display the final, corrected table
print("--- Summary Table with Bonferroni Correction ---")
print(summary_df[[
    "Model A", "Model B", "Dataset", "P-Value", "Significant (Bonferroni)", "Winner (Bonferroni)"
]])



           ANALYSIS WITH BONFERRONI CORRECTION

Original alpha: 0.05
Total number of tests: 196
Bonferroni-corrected alpha: 0.000255

--- Summary Table with Bonferroni Correction ---
       Model A        Model B           Dataset   P-Value  \
0    CliReBERT     CliSciBERT  ClimateInsurance  0.066357   
1    CliReBERT     CliSciBERT         CDPCities  0.000523   
2    CliReBERT     CliSciBERT       ClimateText  0.004932   
3    CliReBERT     CliSciBERT     ClimateStance  0.004913   
4    CliReBERT     CliSciBERT        ClimateEng  0.000148   
..         ...            ...               ...       ...   
191    RoBERTa  DistilRoBERTa       ClimateText  0.452546   
192    RoBERTa  DistilRoBERTa     ClimateStance  0.703328   
193    RoBERTa  DistilRoBERTa        ClimateEng  0.767192   
194    RoBERTa  DistilRoBERTa            SciDCC  0.577573   
195    RoBERTa  DistilRoBERTa      ClimateFEVER  0.009633   

     Significant (Bonferroni) Winner (Bonferroni)  
0                       False 

In [10]:
summary_df

,Model A,Model B,Dataset,P-Value,Significant (p < 0.05),Winner,CliReBERT Avg,CliSciBERT Avg,SciBERT Avg,SciClimateBERT Avg,ClimateBERT Avg,BERT Avg,RoBERTa Avg,DistilRoBERTa Avg,Significant (Bonferroni),Winner (Bonferroni)
0,CliReBERT,CliSciBERT,ClimateInsurance,0.066357,False,None,0.813036,0.801110,NaN,NaN,NaN,NaN,NaN,NaN,False,None
1,CliReBERT,CliSciBERT,CDPCities,0.000523,True,CliSciBERT,0.618628,0.654746,NaN,NaN,NaN,NaN,NaN,NaN,False,None
2,CliReBERT,CliSciBERT,ClimateText,0.004932,True,CliReBERT,0.841904,0.749344,NaN,NaN,NaN,NaN,NaN,NaN,False,None
3,CliReBERT,CliSciBERT,ClimateStance,0.004913,True,CliReBERT,0.502296,0.419006,NaN,NaN,NaN,NaN,NaN,NaN,False,None
4,CliReBERT,CliSciBERT,ClimateEng,0.000148,True,CliReBERT,0.660430,0.486436,NaN,NaN,NaN,NaN,NaN,NaN,True,CliReBERT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,RoBERTa,DistilRoBERTa,ClimateText,0.452546,False,None,NaN,NaN,NaN,NaN,NaN,NaN,0.686196,0.736690,False,None
192,RoBERTa,DistilRoBERTa,ClimateStance,0.703328,False,None,NaN,NaN,NaN,NaN,NaN,NaN,0.395082,0.411152,False,None
193,RoBERTa,DistilRoBERTa,ClimateEng,0.767192,False,None,NaN,NaN,NaN,NaN,NaN,NaN,0.323328,0.353162,False,None
194,RoBERTa,DistilRoBERTa,SciDCC,0.577573,False,None,NaN,NaN,NaN,NaN,NaN,NaN,0.525482,0.527618,False,None


In [18]:
main_model = 'CliReBERT'
other_models = [m for m in model_names if m != main_model]

results_summary_detailed = []

for other_model in other_models:
    # Find all comparisons between the two models
    comparison_df = summary_df[
        ((summary_df['Model A'] == main_model) & (summary_df['Model B'] == other_model)) |
        ((summary_df['Model A'] == other_model) & (summary_df['Model B'] == main_model))
    ]
    
    # Start with all results significant at the p < 0.05 level
    significant_comps = comparison_df[comparison_df['Significant (p < 0.05)']].copy()
    
    # If there are no differences even at the lenient level, we can skip
    if significant_comps.empty:
        wins_str = "—"
        losses_str = "—"
    else:
        # **NEW LOGIC HERE**
        # Create a new column with the dataset name, bolded if it passes Bonferroni
        # We use Markdown's **bold** syntax
        significant_comps['formatted_dataset'] = significant_comps.apply(
            lambda row: f"**{row['Dataset']}**" if row['Significant (Bonferroni)'] else row['Dataset'],
            axis=1
        )

        # Separate into wins and losses based on the 'Winner' column, using the new formatted strings
        wins = significant_comps[significant_comps['Winner'] == main_model]['formatted_dataset'].tolist()
        losses = significant_comps[significant_comps['Winner'] == other_model]['formatted_dataset'].tolist()
        
        wins_str = ', '.join(wins) if wins else "—"
        losses_str = ', '.join(losses) if losses else "—"
        
    results_summary_detailed.append({
        'Comparison': f"{main_model} vs. {other_model}",
        'Significantly Better On': wins_str,
        'Significantly Worse On': losses_str
    })

wins_losses_detailed_df = pd.DataFrame(results_summary_detailed)

# print("--- 'Wins and Losses' Summary Table (with two significance levels) ---")
print(wins_losses_detailed_df.to_markdown(index=False))

| Comparison                   | Significantly Better On                                      | Significantly Worse On   |
|:-----------------------------|:-------------------------------------------------------------|:-------------------------|
| CliReBERT vs. CliSciBERT     | ClimateText, ClimateStance, **ClimateEng**, SciDCC           | CDPCities                |
| CliReBERT vs. SciBERT        | ClimateText, ClimateEng, SciDCC                              | CDPCities                |
| CliReBERT vs. SciClimateBERT | ClimateInsurance, ClimateText, ClimateStance, **ClimateEng** | CDPCities                |
| CliReBERT vs. ClimateBERT    | **ClimateText**                                              | CDPCities                |
| CliReBERT vs. BERT           | ClimateStance, ClimateEng                                    | CDPCities                |
| CliReBERT vs. RoBERTa        | ClimateEng                                                   | CDPCities, ClimateFEVER  |
| CliReBERT vs. 

In [12]:
# --- Step 1: Count wins at both significance levels ---
lenient_wins = summary_df['Winner'].value_counts()
strict_wins = summary_df['Winner (Bonferroni)'].value_counts()

# --- Step 2: Create the summary DataFrame and populate with raw numbers ---
final_summary_df = pd.DataFrame(index=model_names)
final_summary_df.index.name = 'Model'

final_summary_df['lenient_wins_count'] = final_summary_df.index.map(lenient_wins).fillna(0).astype(int)
final_summary_df['strict_wins_count'] = final_summary_df.index.map(strict_wins).fillna(0).astype(int)

# --- Step 3: Calculate the total possible wins and both win rates ---
num_models = len(model_names)
num_datasets = len(dataset_names)
possible_wins_per_model = (num_models - 1) * num_datasets

final_summary_df['lenient_win_rate'] = final_summary_df['lenient_wins_count'] / possible_wins_per_model
final_summary_df['strict_win_rate'] = final_summary_df['strict_wins_count'] / possible_wins_per_model

# --- Step 4: Create the combined string columns for presentation ---
final_summary_df['Significant Wins'] = final_summary_df.apply(
    lambda row: f"{row['lenient_wins_count']} ({row['strict_wins_count']}) / {possible_wins_per_model}",
    axis=1
)

final_summary_df['Win Rate'] = final_summary_df.apply(
    lambda row: f"{row['lenient_win_rate']:.2%} ({row['strict_win_rate']:.2%})",
    axis=1
)

# --- Step 5: Sort by the most important metric (strict wins) and select final columns ---
# THIS IS THE UPDATED LINE:
# Sort by strict wins first, then use lenient wins as a tie-breaker.
final_summary_df = final_summary_df.sort_values(
    by=['strict_wins_count', 'lenient_wins_count'],
    ascending=[False, False]
)

# Select only the beautifully formatted columns for the final display.
display_df = final_summary_df[['Significant Wins', 'Win Rate']]

# --- Display the final table in a clean format ---
# print(f"Total number of models: {num_models}")
# print(f"Total number of datasets: {num_datasets}")
# print(f"Total possible wins per model = ({num_models} - 1) * {num_datasets} = {possible_wins_per_model}\n")

# print("--- Final Model Leaderboard (Multi-level Sort) ---")
print(display_df.to_markdown())

| Model          | Significant Wins   | Win Rate       |
|:---------------|:-------------------|:---------------|
| CliReBERT      | 18.0 (4.0) / 49    | 36.73% (8.16%) |
| ClimateBERT    | 12.0 (1.0) / 49    | 24.49% (2.04%) |
| BERT           | 10.0 (0.0) / 49    | 20.41% (0.00%) |
| RoBERTa        | 9.0 (0.0) / 49     | 18.37% (0.00%) |
| CliSciBERT     | 7.0 (0.0) / 49     | 14.29% (0.00%) |
| SciClimateBERT | 2.0 (0.0) / 49     | 4.08% (0.00%)  |
| DistilRoBERTa  | 2.0 (0.0) / 49     | 4.08% (0.00%)  |
| SciBERT        | 1.0 (0.0) / 49     | 2.04% (0.00%)  |


## Wilcoxon signed-rank test

In [73]:
# Significance level and Bonferroni correction remain the same
alpha = 0.1
total_tests = len(list(combinations(model_names, 2))) * len(dataset_names)
bonferroni_alpha = alpha / total_tests

summary_results = []

# Iterate through all unique pairs of models
for model_a, model_b in combinations(model_names, 2):
    for dataset in dataset_names:
        scores_a = results[model_a][dataset]['seeds']
        scores_b = results[model_b][dataset]['seeds']
        
        avg_a = results[model_a][dataset]['average']
        avg_b = results[model_b][dataset]['average']

        if scores_a == scores_b:
             # If scores are identical, p-value is 1.0 and there's no difference.
            stat, p_value = 0, 1.0
        else:
            stat, p_value = wilcoxon(scores_a, scores_b, zero_method='pratt')
        # -----------------------------------------

        winner = "None"
        winner_bonferroni = "None"
        
        is_significant_lenient = p_value < alpha
        if is_significant_lenient:
            winner = model_a if avg_a > avg_b else model_b
            
        is_significant_bonferroni = p_value < bonferroni_alpha
        if is_significant_bonferroni:
            winner_bonferroni = model_a if avg_a > avg_b else model_b
        
        summary_results.append({
            "Model A": model_a,
            "Model B": model_b,
            "Dataset": dataset,
            "P-Value": p_value,
            "Significant (p < 0.05)": is_significant_lenient,
            "Winner": winner,
            "Significant (Bonferroni)": is_significant_bonferroni,
            "Winner (Bonferroni)": winner_bonferroni,
        })

# Create the new summary_df. This is now based on Wilcoxon p-values.
summary_df = pd.DataFrame(summary_results)

print("summary_df successfully created using Wilcoxon signed-rank test.")
summary_df

summary_df successfully created using Wilcoxon signed-rank test.


/home/p0l3/.local/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/p0l3/.local/lib/python3.10/site-packages/scipy/stats/_morestats.py:3428: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,Model A,Model B,Dataset,P-Value,Significant (p < 0.05),Winner,Significant (Bonferroni),Winner (Bonferroni)
0,CliReBERT,CliSciBERT,ClimateInsurance,0.0625,True,CliReBERT,False,None
1,CliReBERT,CliSciBERT,CDPCities,0.0625,True,CliSciBERT,False,None
2,CliReBERT,CliSciBERT,ClimateText,0.0625,True,CliReBERT,False,None
3,CliReBERT,CliSciBERT,ClimateStance,0.0625,True,CliReBERT,False,None
4,CliReBERT,CliSciBERT,ClimateEng,0.0625,True,CliReBERT,False,None
...,...,...,...,...,...,...,...,...
191,RoBERTa,DistilRoBERTa,ClimateText,0.6250,False,None,False,None
192,RoBERTa,DistilRoBERTa,ClimateStance,0.6250,False,None,False,None
193,RoBERTa,DistilRoBERTa,ClimateEng,0.6250,False,None,False,None
194,RoBERTa,DistilRoBERTa,SciDCC,0.6250,False,None,False,None


## CD-Diagram

In [13]:
# Author: Hassan Ismail Fawaz <hassan.ismail-fawaz@uha.fr>
#         Germain Forestier <germain.forestier@uha.fr>
#         Jonathan Weber <jonathan.weber@uha.fr>
#         Lhassane Idoumghar <lhassane.idoumghar@uha.fr>
#         Pierre-Alain Muller <pierre-alain.muller@uha.fr>
# License: GPL3
STAT_SIG = 1
import numpy as np
import pandas as pd
import matplotlib

matplotlib.use('agg')
import matplotlib.pyplot as plt

matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'

import operator
import math
from scipy.stats import wilcoxon
from scipy.stats import friedmanchisquare
import networkx

# inspired from orange3 https://docs.orange.biolab.si/3/data-mining-library/reference/evaluation.cd.html
def graph_ranks(avranks, names, p_values, cd=None, cdmethod=None, lowv=None, highv=None,
                width=6, textspace=1, reverse=False, filename=None, labels=False, **kwargs):
    """
    Draws a CD graph, which is used to display  the differences in methods'
    performance. See Janez Demsar, Statistical Comparisons of Classifiers over
    Multiple Data Sets, 7(Jan):1--30, 2006.

    Needs matplotlib to work.

    The image is ploted on `plt` imported using
    `import matplotlib.pyplot as plt`.

    Args:
        avranks (list of float): average ranks of methods.
        names (list of str): names of methods.
        cd (float): Critical difference used for statistically significance of
            difference between methods.
        cdmethod (int, optional): the method that is compared with other methods
            If omitted, show pairwise comparison of methods
        lowv (int, optional): the lowest shown rank
        highv (int, optional): the highest shown rank
        width (int, optional): default width in inches (default: 6)
        textspace (int, optional): space on figure sides (in inches) for the
            method names (default: 1)
        reverse (bool, optional):  if set to `True`, the lowest rank is on the
            right (default: `False`)
        filename (str, optional): output file name (with extension). If not
            given, the function does not write a file.
        labels (bool, optional): if set to `True`, the calculated avg rank
        values will be displayed
    """
    try:
        import matplotlib
        import matplotlib.pyplot as plt
        from matplotlib.backends.backend_agg import FigureCanvasAgg
    except ImportError:
        raise ImportError("Function graph_ranks requires matplotlib.")

    width = float(width)
    textspace = float(textspace)

    def nth(l, n):
        """
        Returns only nth elemnt in a list.
        """
        n = lloc(l, n)
        return [a[n] for a in l]

    def lloc(l, n):
        """
        List location in list of list structure.
        Enable the use of negative locations:
        -1 is the last element, -2 second last...
        """
        if n < 0:
            return len(l[0]) + n
        else:
            return n

    def mxrange(lr):
        """
        Multiple xranges. Can be used to traverse matrices.
        This function is very slow due to unknown number of
        parameters.

        >>> mxrange([3,5])
        [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]

        >>> mxrange([[3,5,1],[9,0,-3]])
        [(3, 9), (3, 6), (3, 3), (4, 9), (4, 6), (4, 3)]

        """
        if not len(lr):
            yield ()
        else:
            # it can work with single numbers
            index = lr[0]
            if isinstance(index, int):
                index = [index]
            for a in range(*index):
                for b in mxrange(lr[1:]):
                    yield tuple([a] + list(b))

    def print_figure(fig, *args, **kwargs):
        canvas = FigureCanvasAgg(fig)
        canvas.print_figure(*args, **kwargs)

    sums = avranks

    nnames = names
    ssums = sums

    if lowv is None:
        lowv = min(1, int(math.floor(min(ssums))))
    if highv is None:
        highv = max(len(avranks), int(math.ceil(max(ssums))))

    cline = 0.4

    k = len(sums)

    lines = None

    linesblank = 0
    scalewidth = width - 2 * textspace

    def rankpos(rank):
        if not reverse:
            a = rank - lowv
        else:
            a = highv - rank
        return textspace + scalewidth / (highv - lowv) * a

    distanceh = 0.25

    cline += distanceh

    # calculate height needed height of an image
    minnotsignificant = max(2 * 0.2, linesblank)
    height = cline + ((k + 1) / 2) * 0.2 + minnotsignificant

    fig = plt.figure(figsize=(width, height))
    fig.set_facecolor('white')
    ax = fig.add_axes([0, 0, 1, 1])  # reverse y axis
    ax.set_axis_off()

    hf = 1. / height  # height factor
    wf = 1. / width

    def hfl(l):
        return [a * hf for a in l]

    def wfl(l):
        return [a * wf for a in l]

    # Upper left corner is (0,0).
    ax.plot([0, 1], [0, 1], c="w")
    ax.set_xlim(0, 1)
    ax.set_ylim(1, 0)

    def line(l, color='k', **kwargs):
        """
        Input is a list of pairs of points.
        """
        ax.plot(wfl(nth(l, 0)), hfl(nth(l, 1)), color=color, **kwargs)

    def text(x, y, s, *args, **kwargs):
        ax.text(wf * x, hf * y, s, *args, **kwargs)

    line([(textspace, cline), (width - textspace, cline)], linewidth=2)

    bigtick = 0.3
    smalltick = 0.15
    linewidth = 2.0
    linewidth_sign = 4.0

    tick = None
    for a in list(np.arange(lowv, highv, 0.5)) + [highv]:
        tick = smalltick
        if a == int(a):
            tick = bigtick
        line([(rankpos(a), cline - tick / 2),
              (rankpos(a), cline)],
             linewidth=2)

    for a in range(lowv, highv + 1):
        text(rankpos(a), cline - tick / 2 - 0.05, str(a),
             ha="center", va="bottom", size=16)

    k = len(ssums)

    def filter_names(name):
        return name

    space_between_names = 0.24

    for i in range(math.ceil(k / 2)):
        chei = cline + minnotsignificant + i * space_between_names
        line([(rankpos(ssums[i]), cline),
              (rankpos(ssums[i]), chei),
              (textspace - 0.1, chei)],
             linewidth=linewidth)
        if labels:
            text(textspace + 0.3, chei - 0.075, format(ssums[i], '.4f'), ha="right", va="center", size=10)
        text(textspace - 0.2, chei, filter_names(nnames[i]), ha="right", va="center", size=16)

    for i in range(math.ceil(k / 2), k):
        chei = cline + minnotsignificant + (k - i - 1) * space_between_names
        line([(rankpos(ssums[i]), cline),
              (rankpos(ssums[i]), chei),
              (textspace + scalewidth + 0.1, chei)],
             linewidth=linewidth)
        if labels:
            text(textspace + scalewidth - 0.3, chei - 0.075, format(ssums[i], '.4f'), ha="left", va="center", size=10)
        text(textspace + scalewidth + 0.2, chei, filter_names(nnames[i]),
             ha="left", va="center", size=16)

    # no-significance lines
    def draw_lines(lines, side=0.05, height=0.1):
        start = cline + 0.2

        for l, r in lines:
            line([(rankpos(ssums[l]) - side, start),
                  (rankpos(ssums[r]) + side, start)],
                 linewidth=linewidth_sign)
            start += height
            print('drawing: ', l, r)

    # draw_lines(lines)
    start = cline + 0.2
    side = -0.02
    height = 0.1

    # draw no significant lines
    # get the cliques
    cliques = form_cliques(p_values, nnames)
    i = 1
    achieved_half = False
    print(nnames)
    for clq in cliques:
        if len(clq) == 1:
            continue
        print(clq)
        min_idx = np.array(clq).min()
        max_idx = np.array(clq).max()
        if min_idx >= len(nnames) / 2 and achieved_half == False:
            start = cline + 0.25
            achieved_half = True
        line([(rankpos(ssums[min_idx]) - side, start),
              (rankpos(ssums[max_idx]) + side, start)],
             linewidth=linewidth_sign)
        start += height


def form_cliques(p_values, nnames):
    """
    This method forms the cliques
    """
    # first form the numpy matrix data
    m = len(nnames)
    g_data = np.zeros((m, m), dtype=np.int64)
    for p in p_values:
        if p[3] == False:
            i = np.where(nnames == p[0])[0][0]
            j = np.where(nnames == p[1])[0][0]
            min_i = min(i, j)
            max_j = max(i, j)
            g_data[min_i, max_j] = 1

    g = networkx.Graph(g_data)
    return networkx.find_cliques(g)


def draw_cd_diagram(df_perf=None, alpha=0.05, title=None, labels=False):
    """
    Draws the critical difference diagram given the list of pairwise classifiers that are
    significant or not
    """
    p_values, average_ranks, _ = wilcoxon_holm(df_perf=df_perf, alpha=alpha)

    print(average_ranks)

    for p in p_values:
        print(p)


    graph_ranks(average_ranks.values, average_ranks.keys(), p_values,
                cd=None, reverse=True, width=9, textspace=1.5, labels=labels)

    font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 10,
        }
    sets_list = df_perf["dataset_name"].unique().tolist()
    sets_list.sort
    sets = "_".join(sets_list)
    if title:
        # plt.title(title+"-"+sets,fontdict=font, y=0.9, x=0.5)
        plt.title("Critical Difference Diagram", size=18)
    # sets = df_perf.unique()
    
    plt.savefig(f'./ALL/{len(sets_list)}_{STAT_SIG}_cd_diagram_{sets}.png',bbox_inches='tight')

def wilcoxon_holm(alpha=0.05, df_perf=None):
    """
    Applies the wilcoxon signed rank test between each pair of algorithm and then use Holm
    to reject the null's hypothesis
    """
    global STAT_SIG
    STAT_SIG = 1
    print(pd.unique(df_perf['model_name']))
    # count the number of tested datasets per classifier
    df_counts = pd.DataFrame({'count': df_perf.groupby(
        ['model_name']).size()}).reset_index()
    # get the maximum number of tested datasets
    max_nb_datasets = df_counts['count'].max()
    # get the list of classifiers who have been tested on nb_max_datasets
    classifiers = list(df_counts.loc[df_counts['count'] == max_nb_datasets]
                       ['model_name'])
    # test the null hypothesis using friedman before doing a post-hoc analysis
    friedman_p_value = friedmanchisquare(*(
        np.array(df_perf.loc[df_perf['model_name'] == c]['avg_macro_f1'])
        for c in classifiers))[1]
    if friedman_p_value >= alpha:
        # then the null hypothesis over the entire classifiers cannot be rejected
        STAT_SIG = 0
        print('the null hypothesis over the entire classifiers cannot be rejected')
        # exit()
    
    # get the number of classifiers
    m = len(classifiers)
    # init array that contains the p-values calculated by the Wilcoxon signed rank test
    p_values = []
    # loop through the algorithms to compare pairwise
    for i in range(m - 1):
        # get the name of classifier one
        classifier_1 = classifiers[i]
        # get the performance of classifier one
        perf_1 = np.array(df_perf.loc[df_perf['model_name'] == classifier_1]['avg_macro_f1']
                          , dtype=np.float64)
        for j in range(i + 1, m):
            # get the name of the second classifier
            classifier_2 = classifiers[j]
            # get the performance of classifier one
            perf_2 = np.array(df_perf.loc[df_perf['model_name'] == classifier_2]
                              ['avg_macro_f1'], dtype=np.float64)
            # calculate the p_value
            p_value = wilcoxon(perf_1, perf_2, zero_method='pratt')[1]
            # appen to the list
            p_values.append((classifier_1, classifier_2, p_value, False))
    # get the number of hypothesis
    k = len(p_values)
    # sort the list in acsending manner of p-value
    p_values.sort(key=operator.itemgetter(2))

    # loop through the hypothesis
    for i in range(k):
        # correct alpha with holm
        new_alpha = float(alpha / (k - i))
        # test if significant after holm's correction of alpha
        if p_values[i][2] <= new_alpha:
            p_values[i] = (p_values[i][0], p_values[i][1], p_values[i][2], True)
        else:
            # stop
            break
    # compute the average ranks to be returned (useful for drawing the cd diagram)
    # sort the dataframe of performances
    sorted_df_perf = df_perf.loc[df_perf['model_name'].isin(classifiers)]. \
        sort_values(['model_name', 'dataset_name'])
    # get the rank data
    rank_data = np.array(sorted_df_perf['avg_macro_f1']).reshape(m, max_nb_datasets)

    # create the data frame containg the accuracies
    df_ranks = pd.DataFrame(data=rank_data, index=np.sort(classifiers), columns=
    np.unique(sorted_df_perf['dataset_name']))

    # number of wins
    dfff = df_ranks.rank(ascending=False)
    print(dfff[dfff == 1.0].sum(axis=1))

    # average the ranks
    average_ranks = df_ranks.rank(ascending=False).mean(axis=1).sort_values(ascending=False)
    # return the p-values and the average ranks
    return p_values, average_ranks, max_nb_datasets


df_perf_ = pd.read_csv('CLIMABENCH_SUMMARY.csv', index_col=False)

# DistilRoBERTas ["ClimateBERT", "SciClimateBERT", "DistilRoBERTa"]
# BERTs ["CliReBERT", "CliSciBERT", "SciBERT", "BERT"]
# RoBERTa ["RoBERTa"]
# Top 3 
# All ["CliReBERT", "CliSciBERT", "SciBERT", "BERT", "ClimateBERT", "SciClimateBERT", "DistilRoBERTa", "RoBERTa"]
df_perf = df_perf_[df_perf_.model_name.isin(["CliReBERT", "CliSciBERT", "SciBERT", "BERT", "ClimateBERT", "SciClimateBERT", "DistilRoBERTa", "RoBERTa"])]

# print(df_perf.columns)

def combs(a):
    if len(a) == 0:
        return [[]]
    cs = []
    for c in combs(a[1:]):
        cs += [c, c+[a[0]]]
    return cs

from tqdm import tqdm

## For al combinations
for df_comb in tqdm(combs(df_perf["dataset_name"][:7].to_list())):
    df_perf_sub = df_perf[df_perf["dataset_name"].isin(df_comb)]
    if df_perf_sub.empty:
        continue
    else:
        draw_cd_diagram(df_perf=df_perf_sub, title='avg_macro_f1', labels=True)

# # Single Image

draw_cd_diagram(df_perf=df_perf, title='avg_macro_f1', labels=True)

  2%|▏         | 2/128 [00:00<00:13,  9.64it/s]

['BERT' 'ClimateBERT' 'CliReBERT' 'CliSciBERT' 'DistilRoBERTa' 'RoBERTa'
 'SciBERT' 'SciClimateBERT']
the null hypothesis over the entire classifiers cannot be rejected
BERT              0.0
CliReBERT         0.0
CliSciBERT        0.0
ClimateBERT       0.0
DistilRoBERTa     0.0
RoBERTa           1.0
SciBERT           0.0
SciClimateBERT    0.0
dtype: float64
CliReBERT         8.0
DistilRoBERTa     7.0
SciBERT           6.0
ClimateBERT       5.0
BERT              4.0
SciClimateBERT    3.0
CliSciBERT        2.0
RoBERTa           1.0
dtype: float64
('BERT', 'CliReBERT', 1.0, False)
('BERT', 'CliSciBERT', 1.0, False)
('BERT', 'ClimateBERT', 1.0, False)
('BERT', 'DistilRoBERTa', 1.0, False)
('BERT', 'RoBERTa', 1.0, False)
('BERT', 'SciBERT', 1.0, False)
('BERT', 'SciClimateBERT', 1.0, False)
('CliReBERT', 'CliSciBERT', 1.0, False)
('CliReBERT', 'ClimateBERT', 1.0, False)
('CliReBERT', 'DistilRoBERTa', 1.0, False)
('CliReBERT', 'RoBERTa', 1.0, False)
('CliReBERT', 'SciBERT', 1.0, False)
('Cli

  2%|▏         | 3/128 [00:00<00:19,  6.48it/s]

['BERT' 'ClimateBERT' 'CliReBERT' 'CliSciBERT' 'DistilRoBERTa' 'RoBERTa'
 'SciBERT' 'SciClimateBERT']
the null hypothesis over the entire classifiers cannot be rejected
BERT              0.0
CliReBERT         1.0
CliSciBERT        0.0
ClimateBERT       0.0
DistilRoBERTa     0.0
RoBERTa           1.0
SciBERT           0.0
SciClimateBERT    0.0
dtype: float64
DistilRoBERTa     6.5
SciBERT           5.5
SciClimateBERT    5.5
CliReBERT         4.5
BERT              4.0
RoBERTa           4.0
ClimateBERT       3.5
CliSciBERT        2.5
dtype: float64
('BERT', 'CliSciBERT', 0.5, False)
('BERT', 'DistilRoBERTa', 0.5, False)
('BERT', 'SciBERT', 0.5, False)
('CliSciBERT', 'DistilRoBERTa', 0.5, False)
('CliSciBERT', 'SciBERT', 0.5, False)
('CliSciBERT', 'SciClimateBERT', 0.5, False)
('ClimateBERT', 'DistilRoBERTa', 0.5, False)
('ClimateBERT', 'SciBERT', 0.5, False)
('DistilRoBERTa', 'SciBERT', 0.5, False)
('RoBERTa', 'SciClimateBERT', 0.5, False)
('BERT', 'CliReBERT', 1.0, False)
('BERT', 'Climat

  3%|▎         | 4/128 [00:00<00:21,  5.66it/s]

...

BERT              0.0
CliReBERT         2.0
CliSciBERT        0.0
ClimateBERT       2.0
DistilRoBERTa     0.0
RoBERTa           1.0
SciBERT           0.0
SciClimateBERT    0.0
dtype: float64
RoBERTa           6.0
CliSciBERT        5.8
SciBERT           5.2
SciClimateBERT    5.2
DistilRoBERTa     4.6
ClimateBERT       3.6
BERT              2.8
CliReBERT         2.8
dtype: float64
('BERT', 'CliSciBERT', 0.0625, False)
('BERT', 'SciClimateBERT', 0.0625, False)
('CliReBERT', 'CliSciBERT', 0.0625, False)
('BERT', 'DistilRoBERTa', 0.125, False)
('CliReBERT', 'DistilRoBERTa', 0.125, False)
('CliReBERT', 'SciClimateBERT', 0.125, False)
('BERT', 'RoBERTa', 0.3125, False)
('BERT', 'SciBERT', 0.3125, False)
('CliReBERT', 'RoBERTa', 0.3125, False)
('CliReBERT', 'SciBERT', 0.3125, False)
('CliSciBERT', 'ClimateBERT', 0.3125, False)
('ClimateBERT', 'DistilRoBERTa', 0.3125, False)
('ClimateBERT', 'RoBERTa', 0.3125, False)
('BERT', 'CliReBERT', 0.4375, False)
('ClimateBERT', 'SciBERT', 0.4375, False)


 99%|█████████▉| 127/128 [00:23<00:00,  5.92it/s]

['BERT' 'ClimateBERT' 'CliReBERT' 'CliSciBERT' 'DistilRoBERTa' 'RoBERTa'
 'SciBERT' 'SciClimateBERT']
the null hypothesis over the entire classifiers cannot be rejected
BERT              0.0
CliReBERT         3.0
CliSciBERT        0.0
ClimateBERT       2.0
DistilRoBERTa     0.0
RoBERTa           1.0
SciBERT           0.0
SciClimateBERT    0.0
dtype: float64
RoBERTa           6.166667
SciClimateBERT    5.666667
CliSciBERT        5.333333
SciBERT           5.166667
DistilRoBERTa     4.833333
ClimateBERT       3.333333
BERT              3.000000
CliReBERT         2.500000
dtype: float64
('BERT', 'SciClimateBERT', 0.03125, False)
('CliReBERT', 'CliSciBERT', 0.03125, False)
('BERT', 'DistilRoBERTa', 0.0625, False)
('CliReBERT', 'DistilRoBERTa', 0.0625, False)
('CliReBERT', 'SciClimateBERT', 0.0625, False)
('BERT', 'RoBERTa', 0.15625, False)
('BERT', 'SciBERT', 0.15625, False)
('CliReBERT', 'RoBERTa', 0.15625, False)
('CliReBERT', 'SciBERT', 0.15625, False)
('CliSciBERT', 'ClimateBERT', 0.15

100%|██████████| 128/128 [00:24<00:00,  5.33it/s]


Index(['RoBERTa', 'SciBERT', 'SciClimateBERT', 'DistilRoBERTa', 'CliSciBERT',
       'ClimateBERT', 'CliReBERT', 'BERT'],
      dtype='object')
[0, 1, 2, 3, 4, 5, 6, 7]
['BERT' 'ClimateBERT' 'CliReBERT' 'CliSciBERT' 'DistilRoBERTa' 'RoBERTa'
 'SciBERT' 'SciClimateBERT']
the null hypothesis over the entire classifiers cannot be rejected
BERT              0.0
CliReBERT         3.0
CliSciBERT        0.0
ClimateBERT       2.0
DistilRoBERTa     0.0
RoBERTa           2.0
SciBERT           0.0
SciClimateBERT    0.0
dtype: float64
RoBERTa           5.428571
SciBERT           5.285714
SciClimateBERT    5.285714
DistilRoBERTa     5.142857
CliSciBERT        4.857143
ClimateBERT       3.571429
CliReBERT         3.285714
BERT              3.142857
dtype: float64
('BERT', 'DistilRoBERTa', 0.03125, False)
('BERT', 'SciClimateBERT', 0.046875, False)
('BERT', 'SciBERT', 0.109375, False)
('CliReBERT', 'CliSciBERT', 0.109375, False)
('CliReBERT', 'DistilRoBERTa', 0.15625, False)
('CliReBERT', 'SciClimate